In [1]:
import pandas as pd

<h2>AQI Data</h2>
The AQI data files is downloaded from https://aqs.epa.gov/aqsweb/airdata/download_files.html, a website of historical data from 1980 to 2020 for Air Quality data, plus some site data to help identify location of monitoring locations.  This data measures both natural and pollution-related air quality, providing a record over time of these measurements.  The data can be downloaded in .csv files, the preferred method for distribution for those situations where live data is not necessary.  

The files contain once-daily measurements of air quality, identified by state, county and site.  Since several measures are possible (ozone, SO2, CO, NO2 and particulates) the measurement type is identified as well. This project includes data from 2010 to 2020, and a file of site data to help locate the measurement sites by latitude and longitude on maps.  Since we are only interested in California data, all files were cleaned to only that data.

<h3>Sites Data</h3>

In [2]:
# Process the sites data file
sites_file = 'Data/aqs_sites.csv'
df = pd.read_csv(sites_file)
df.head()

,State Code,County Code,Site Number,Latitude,Longitude,Datum,Elevation,Land Use,Location Setting,Site Established Date,...,Owning Agency,Local Site Name,Address,Zip Code,State Name,County Name,City Name,CBSA Name,Tribe Name,Extraction Date
0,01,1,1,32.437458,-86.472891,WGS84,64.0,RESIDENTIAL,SUBURBAN,1974-05-01,...,Al Dept Of Env Mgt,NaN,"KING ARTHUR TRAILER COURT, PRATTVILLE,AL",36067.0,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2020-11-24
1,01,1,2,32.428470,-86.443585,WGS84,0.0,AGRICULTURAL,RURAL,1980-01-01,...,Al Dept Of Env Mgt,NaN,COUNTY RD 4 PRATTVILLE EXPERIMENT ST,NaN,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2020-11-24
2,01,1,3,32.332659,-86.791521,WGS84,41.0,FOREST,RURAL,1989-08-31,...,Al Dept Of Env Mgt,NaN,"1170 COUNTY RD.15 SO., SELMA, AL. 36701",36003.0,Alabama,Autauga,Not in a City,"Montgomery, AL",NaN,2020-11-24
3,01,3,1,0.000000,0.000000,NAD27,0.0,UNKNOWN,RURAL,1959-01-01,...,US EPA National Exposure Research Lab,NaN,CHANDLER RESIDENCE FT MORGAN STATE PARK,NaN,Alabama,Baldwin,Not in a city,"Daphne-Fairhope-Foley, AL",NaN,2020-11-24
4,01,3,2,30.552367,-87.706911,WGS84,0.0,COMMERCIAL,RURAL,1974-01-01,...,Al Dept Of Env Mgt,NaN,COR PENNSYLVANIA & CHICAGO (ROBERTSDALE),36567.0,Alabama,Baldwin,Robertsdale,"Daphne-Fairhope-Foley, AL",NaN,2020-11-24


In [3]:
#Create new column "Defining Site" to match to AQI files The format of defining site is xx-xxx-xxxx
df['Defining Site'] = df['State Code'].apply(lambda x: '{0:0>2}'.format(x)) + '-' +  \
    df['County Code'].apply(lambda y: '{0:0>3}'.format(y)) + '-' + \
    df['Site Number'].apply(lambda z: '{0:0>4}'.format(z))
df.head()

,State Code,County Code,Site Number,Latitude,Longitude,Datum,Elevation,Land Use,Location Setting,Site Established Date,...,Local Site Name,Address,Zip Code,State Name,County Name,City Name,CBSA Name,Tribe Name,Extraction Date,Defining Site
0,01,1,1,32.437458,-86.472891,WGS84,64.0,RESIDENTIAL,SUBURBAN,1974-05-01,...,NaN,"KING ARTHUR TRAILER COURT, PRATTVILLE,AL",36067.0,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2020-11-24,01-001-0001
1,01,1,2,32.428470,-86.443585,WGS84,0.0,AGRICULTURAL,RURAL,1980-01-01,...,NaN,COUNTY RD 4 PRATTVILLE EXPERIMENT ST,NaN,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2020-11-24,01-001-0002
2,01,1,3,32.332659,-86.791521,WGS84,41.0,FOREST,RURAL,1989-08-31,...,NaN,"1170 COUNTY RD.15 SO., SELMA, AL. 36701",36003.0,Alabama,Autauga,Not in a City,"Montgomery, AL",NaN,2020-11-24,01-001-0003
3,01,3,1,0.000000,0.000000,NAD27,0.0,UNKNOWN,RURAL,1959-01-01,...,NaN,CHANDLER RESIDENCE FT MORGAN STATE PARK,NaN,Alabama,Baldwin,Not in a city,"Daphne-Fairhope-Foley, AL",NaN,2020-11-24,01-003-0001
4,01,3,2,30.552367,-87.706911,WGS84,0.0,COMMERCIAL,RURAL,1974-01-01,...,NaN,COR PENNSYLVANIA & CHICAGO (ROBERTSDALE),36567.0,Alabama,Baldwin,Robertsdale,"Daphne-Fairhope-Foley, AL",NaN,2020-11-24,01-003-0002


In [4]:
# Reorganize the columns, drop the three components of defining site, and reindex to use the Defining Site.
columns = ['Defining Site', 'Latitude', 'Longitude', 'Elevation', 'Land Use',
                    'Location Setting', 'State Name', 'County Name', 'City Name', 'CBSA Name']
sites_df = df[columns]
# Create California only dataset
ca_sites_df = sites_df.loc[sites_df["State Name"]=='California']
ca_sites_df.head()

,Defining Site,Latitude,Longitude,Elevation,Land Use,Location Setting,State Name,County Name,City Name,CBSA Name
1052,06-001-0001,37.876870,-122.266913,15.0,RESIDENTIAL,URBAN AND CENTER CITY,California,Alameda,Berkeley,"San Francisco-Oakland-Hayward, CA"
1053,06-001-0002,37.533243,-121.958813,22.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,Fremont,"San Francisco-Oakland-Hayward, CA"
1054,06-001-0003,37.684900,-121.765900,150.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,Livermore,"San Francisco-Oakland-Hayward, CA"
1055,06-001-0004,37.800484,-122.266358,38.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,Oakland,"San Francisco-Oakland-Hayward, CA"
1056,06-001-0005,37.798600,-122.268400,10.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,Oakland,"San Francisco-Oakland-Hayward, CA"


In [5]:
# save as ca_sites.csv
exportFile = 'Data/ca_sites.csv'
ca_sites_df.to_csv(exportFile, encoding='utf-8', index=False)


<h3>AQI Data</h3>
The AQI data files are posted by year; to obtain data for 2010 through 2020, 11 files were downloaded and processed to:
<ul>
    <li>Limit to California data</li>
    <li>Assure only records with complete data were included</li>
    <li>Strip away unnecessary columns
</ul>
Since we need several years' data, we will process all the files and merge all years into one data file that can be loaded into Postgres.

In [6]:
# Create list of files to process. 
file_loc = 'Data/daily_aqi_by_county_'
# Create empty dataframe to hold the data
column_names = ["State Name", "county Name", "Date", "AQI", "Category", "Defining Parameter", "Defining Site"]
aqi_df = pd.DataFrame(columns = column_names)

# Since the file names are the same, only the year changes, a for loop should work. 
# Read the data in and process it to add to the dataframe.
for i in range(2010,2021):
    file = file_loc + str(i) + '.csv'
    aqi_data = pd.read_csv(file)
    # need to get only california
    ca_aqi_df = aqi_data.loc[aqi_data["State Name"]=='California']
    # limit to just the columns we want
    ca_limit_df = ca_aqi_df[column_names]
    # append to aqi_df
    aqi_df = aqi_df.append(ca_limit_df, ignore_index=True)
    print("Finished processing " + file)


FileNotFoundError: [Errno 2] File Data/daily_aqi_by_county_2010.csv does not exist: 'Data/daily_aqi_by_county_2010.csv'

In [7]:
# rows without some info are not useful to us, so get rid of them.
aqi_df = aqi_df.dropna()

In [8]:
# Save to csv for future visualization.
aqi_df.to_csv("Data/aqi_full_data.csv")